# 03) 멀티오믹스 Cis-correlation 분석

**핵심 개념**: 같은 유전자의 mRNA 발현과 프로모터 메틸화 간 상관분석 (cis = 같은 유전자)

## 배경: Cis-correlation이란?

| 패턴 | 메틸화 위치 | 발현 관계 | 생물학적 의미 |
|------|-----------|----------|-------------|
| **프로모터 과메틸화** | TSS200, TSS1500, 5'UTR | **음의 상관** (ρ < 0) | 종양억제유전자 침묵 |
| **유전자체 메틸화** | Gene Body, 3'UTR | **양의 상관** (ρ > 0) | 활발한 전사 표지 |

### Trans vs Cis
| 방법 | 비교 대상 | 검정 수 | 문제점 |
|------|----------|---------|--------|
| Trans (기존) | 모든 유전자 × 모든 프로브 | ~수십만 개 | 다중검정 부담, 해석 어려움 |
| **Cis (본 분석)** | **같은 유전자**의 발현 vs 메틸화 | ~1,000개 | 검정 수 감소 → 통계적 파워 ↑ |

## 라이브러리 및 환경 설정

In [ ]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from scipy.stats import spearmanr, mannwhitneyu, kruskal
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
import warnings; warnings.filterwarnings('ignore')

import plot_config
plot_config.setup()
np.random.seed(42)

# 유틸리티: p-value → 유의성 별표
def sig_stars(p):
    if p < 0.001: return '***'
    if p < 0.01:  return '**'
    if p < 0.05:  return '*'
    return ''

## 1. 데이터 로드

| 파일 | 내용 | 형태 |
|------|------|------|
| `input/mrna_matched.txt` | mRNA 발현 | gene × sample |
| `input/methylation_gene_level.txt` | **유전자별 메틸화** (TSS 프로브 우선 집계) | gene × sample |
| `input/probe_gene_mapping.tsv` | 프로브→유전자 매핑 | probe, gene, location |
| `input/clinical_matched.tsv` | 임상 정보 | patient × variable |

In [ ]:
# 파일 경로
MRNA_FILE  = "input/mrna_matched.txt"
CLIN_FILE  = "input/clinical_matched.tsv"
METH_FILE  = "input/methylation_gene_level.txt"
PROBE_FILE = "input/probe_gene_mapping.tsv"
SAMPLE_COL = "Sample ID"
RESULT_DIR = "results"

# 데이터 로드
mrna      = pd.read_csv(MRNA_FILE,  sep="\t", index_col=0)
clinical  = pd.read_csv(CLIN_FILE,  sep="\t")
meth_gene = pd.read_csv(METH_FILE,  sep="\t", index_col=0)
probe_info = pd.read_csv(PROBE_FILE, sep="\t")

# 중복 유전자 제거: 같은 유전자가 여러 행이면 분산이 가장 큰 행만 유지
if mrna.index.duplicated().any():
    mrna['_var'] = mrna.var(axis=1)
    mrna = mrna.sort_values('_var', ascending=False)
    mrna = mrna[~mrna.index.duplicated(keep='first')]
    mrna = mrna.drop(columns='_var')

# 공통 샘플 매칭
samples = sorted(set(mrna.columns) & set(meth_gene.columns) & set(clinical[SAMPLE_COL]))
mrna     = mrna[samples]
meth_gene = meth_gene[samples]
clinical = clinical[clinical[SAMPLE_COL].isin(samples)].reset_index(drop=True)

# 공통 유전자 매칭 (cis-correlation 대상)
common_genes = sorted(set(mrna.index) & set(meth_gene.index))
mrna_cis = mrna.loc[common_genes, samples]
meth_cis = meth_gene.loc[common_genes, samples]

# 유전자별 메틸화 프로브 위치 정보
gene_locations = probe_info.groupby('gene')['loc_category'].first()
gene_locations = gene_locations.reindex(common_genes).fillna('Other')

print(f"mRNA: {mrna.shape[0]:,}개 유전자 × {mrna.shape[1]}명")
print(f"Methylation: {meth_gene.shape[0]:,}개 유전자")
print(f"공통 유전자 (cis 대상): {len(common_genes):,}개")
print(f"  Promoter: {(gene_locations=='Promoter').sum()}"
      f" | Body: {(gene_locations=='Body').sum()}"
      f" | Other: {(gene_locations=='Other').sum()}")

In [ ]:
# 프로브-유전자 매핑 통계 시각화
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# 1) 위치 카테고리 분포
loc_counts = probe_info['loc_category'].value_counts()
axes[0].bar(loc_counts.index, loc_counts.values,
            color=['#e74c3c', '#2ecc71', '#95a5a6'], alpha=0.8)
axes[0].set_title('프로브 위치 분포')
axes[0].set_ylabel('프로브 수')

# 2) 유전자당 프로브 수
probes_per_gene = probe_info.groupby('gene').size()
axes[1].hist(probes_per_gene.clip(upper=20), bins=20,
             color='steelblue', alpha=0.7, edgecolor='white')
axes[1].set_xlabel('유전자당 프로브 수')
axes[1].set_ylabel('유전자 수')
axes[1].set_title(f'유전자당 프로브 분포 (중앙값: {probes_per_gene.median():.0f}개)')

# 3) 공통 유전자 현황
n_mrna_only = len(set(mrna.index) - set(meth_gene.index))
n_meth_only = len(set(meth_gene.index) - set(mrna.index))
n_common = len(common_genes)
labels = [f'mRNA only\n{n_mrna_only}', f'공통\n{n_common}', f'Meth only\n{n_meth_only}']
axes[2].bar(labels, [n_mrna_only, n_common, n_meth_only],
            color=['steelblue', '#2ecc71', 'coral'], alpha=0.8)
axes[2].set_title('mRNA ∩ Methylation 유전자')
axes[2].set_ylabel('유전자 수')

plt.tight_layout()
plt.show()

### Vibe Coding 미션 D1: 프로브-유전자 매핑 탐색
**미션**: 프로브-유전자 매핑 데이터를 탐색하고 시각화해보세요!

**결과물**: Top 10 유전자의 프로브 수 막대그래프

**도전 과제**:
1. 프로브가 가장 많은 **Top 10 유전자**를 막대그래프로 시각화하기
2. 프로모터 프로브만 가진 유전자 vs Body 프로브만 가진 유전자 수 비교
3. (도전) 특정 유전자(예: TP53, BRCA1)의 프로브 위치 분포 확인

**힌트**: `probe_info.groupby('gene').size().nlargest(10).plot.bar()`

In [ ]:
# 미션 D1 코드를 여기에 작성하세요!

## 2. 단일 유전자 Cis-correlation 예시

**Cis-correlation**: 같은 유전자의 mRNA 발현 ↔ 프로모터 메틸화 간 Spearman 상관

In [ ]:
# 첫 번째 공통 유전자로 시연
gene_ex = common_genes[0]
expr_vals = mrna_cis.loc[gene_ex].values.astype(float)
meth_vals = meth_cis.loc[gene_ex].values.astype(float)
valid = ~(np.isnan(expr_vals) | np.isnan(meth_vals))

rho, pval = spearmanr(expr_vals[valid], meth_vals[valid])

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(expr_vals[valid], meth_vals[valid], alpha=0.6, s=50,
           color='steelblue', edgecolors='white')

# 추세선
z = np.polyfit(expr_vals[valid], meth_vals[valid], 1)
x_line = np.linspace(expr_vals[valid].min(), expr_vals[valid].max(), 100)
ax.plot(x_line, np.poly1d(z)(x_line), 'r--', alpha=0.7, lw=2)

loc_label = gene_locations[gene_ex]
ax.set_xlabel(f"{gene_ex} mRNA 발현량", fontsize=12)
ax.set_ylabel(f"{gene_ex} 메틸화 (β값)", fontsize=12)
ax.set_title(f"Cis-correlation: {gene_ex} ({loc_label})\n"
             f"Spearman ρ = {rho:.3f}, p = {pval:.2e}", fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"유전자: {gene_ex} | 위치: {loc_label}")
print(f"Spearman ρ = {rho:.3f}, p = {pval:.2e}")
if rho < -0.3:
    print("→ 음의 상관: 메틸화↑ → 발현↓ (프로모터 침묵)")
elif rho > 0.3:
    print("→ 양의 상관: 메틸화↑ → 발현↑ (유전자체 활성)")
else:
    print("→ 약한 상관")

## 3. 전체 유전자 Cis-correlation + FDR 보정

모든 공통 유전자에 대해 **Spearman cis-correlation** 계산 후 **Benjamini-Hochberg FDR** 보정

In [ ]:
# 전체 유전자에 대해 cis-correlation 계산
print(f"{len(common_genes):,}개 유전자 cis-correlation 계산 중...")

cis_results = []
for gene in common_genes:
    # 같은 유전자의 발현값과 메틸화값 추출
    expr = mrna_cis.loc[gene].values.astype(float)
    meth = meth_cis.loc[gene].values.astype(float)
    
    # 결측치가 없는 샘플만 사용
    valid = ~(np.isnan(expr) | np.isnan(meth))
    
    if valid.sum() > 10:
        rho, pval = spearmanr(expr[valid], meth[valid])
    else:
        rho, pval = np.nan, np.nan
    
    cis_results.append({
        'gene': gene,
        'rho': rho,
        'p_value': pval,
        'location': gene_locations[gene],
        'n_valid': int(valid.sum())
    })

cis_df = pd.DataFrame(cis_results).dropna(subset=['rho'])

# FDR 보정 (다중검정 보정)
reject, fdr_vals, _, _ = multipletests(cis_df['p_value'].values,
                                        alpha=0.05, method='fdr_bh')
cis_df['fdr'] = fdr_vals
cis_df['significant'] = reject
cis_df = cis_df.sort_values('p_value')

# 결과 요약
n_sig = cis_df['significant'].sum()
n_neg = ((cis_df['rho'] < 0) & cis_df['significant']).sum()
n_pos = ((cis_df['rho'] > 0) & cis_df['significant']).sum()
print(f"\n유의한 유전자 (FDR < 0.05): {n_sig:,}/{len(cis_df):,}개")
print(f"  음의 상관 (메틸화↑ → 발현↓): {n_neg:,}개")
print(f"  양의 상관 (메틸화↑ → 발현↑): {n_pos:,}개")

In [ ]:
# Cis-correlation 분포 + Volcano Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# (1) ρ 분포 히스토그램
sns.histplot(cis_df['rho'], bins=40, kde=True, color='skyblue', alpha=0.7, ax=axes[0])
axes[0].axvline(0, color='gray', ls='-', alpha=0.5)
axes[0].axvline(-0.3, color='red', ls='--', lw=2, alpha=0.7, label='ρ = ±0.3')
axes[0].axvline(0.3, color='green', ls='--', lw=2, alpha=0.7)
axes[0].set_xlabel("Spearman ρ")
axes[0].set_ylabel("유전자 수")
axes[0].set_title(f"Cis-correlation 분포 (n={len(cis_df):,})")
axes[0].legend()

# (2) Volcano Plot: x축=ρ, y축=-log10(FDR)
neg_log_fdr = -np.log10(np.clip(cis_df['fdr'].values, 1e-300, 1))
is_sig = cis_df['significant'].values
rho_vals = cis_df['rho'].values

# 비유의 / 음의 상관 / 양의 상관으로 색 구분
axes[1].scatter(rho_vals[~is_sig], neg_log_fdr[~is_sig],
                alpha=0.4, s=20, c='grey', label=f'비유의 ({(~is_sig).sum():,})')
axes[1].scatter(rho_vals[is_sig & (rho_vals < 0)], neg_log_fdr[is_sig & (rho_vals < 0)],
                alpha=0.6, s=30, c='blue', label=f'음의 상관 ({(is_sig & (rho_vals < 0)).sum():,})')
axes[1].scatter(rho_vals[is_sig & (rho_vals > 0)], neg_log_fdr[is_sig & (rho_vals > 0)],
                alpha=0.6, s=30, c='red', label=f'양의 상관 ({(is_sig & (rho_vals > 0)).sum():,})')

axes[1].axhline(-np.log10(0.05), ls='--', c='orange', alpha=0.5, label='FDR = 0.05')
axes[1].set_xlabel("Spearman ρ")
axes[1].set_ylabel("-log₁₀(FDR)")
axes[1].set_title("Volcano Plot")
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()

## 4. 프로모터 vs Body 위치별 비교

프로모터(TSS) 메틸화 → **발현 억제** (음의 상관)
유전자체(Body) 메틸화 → **전사 활성** (양의 상관)

In [ ]:
# 위치별 cis-correlation 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
loc_order = ['Promoter', 'Body', 'Other']
colors = ['#e74c3c', '#2ecc71', '#95a5a6']

# (1) 위치별 ρ 박스플롯
sns.boxplot(data=cis_df[cis_df['location'].isin(loc_order)],
            x='location', y='rho', order=loc_order,
            palette=dict(zip(loc_order, colors)), ax=axes[0])
axes[0].axhline(0, ls='--', c='gray', alpha=0.5)
axes[0].set_xlabel('메틸화 위치')
axes[0].set_ylabel('Spearman ρ')
axes[0].set_title('위치별 Cis-correlation')

# Promoter vs Body 통계 검정
prom = cis_df[cis_df['location'] == 'Promoter']['rho'].dropna()
body = cis_df[cis_df['location'] == 'Body']['rho'].dropna()
if len(prom) > 5 and len(body) > 5:
    _, mw_p = mannwhitneyu(prom, body, alternative='two-sided')
    axes[0].text(0.5, 0.95, f'Promoter vs Body: p={mw_p:.2e}',
                 transform=axes[0].transAxes, ha='center', va='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# (2) 위치별 유의 비율
sig_counts = cis_df.groupby('location').agg(
    total=('gene', 'count'),
    n_sig=('significant', 'sum')
).reindex(loc_order).dropna()
sig_counts['pct'] = sig_counts['n_sig'] / sig_counts['total'] * 100

axes[1].bar(sig_counts.index, sig_counts['pct'], color=colors, alpha=0.8)
for i, (_, row) in enumerate(sig_counts.iterrows()):
    axes[1].text(i, row['pct'] + 1,
                 f"{row['n_sig']:.0f}/{row['total']:.0f}",
                 ha='center', fontsize=10)
axes[1].set_ylabel('유의한 유전자 비율 (%)')
axes[1].set_title('위치별 유의 비율 (FDR < 0.05)')

plt.tight_layout()
plt.show()

### Vibe Coding 미션 D2: 위치별 상관 패턴 시각화
**미션**: Promoter와 Body 메틸화의 차이를 시각적으로 비교해보세요!

**결과물**: 위치별 ρ 분포를 겹쳐 그린 KDE 그래프

**도전 과제**:
1. Promoter 유전자 중 **가장 강한 음의 상관 Top 5** 출력
2. Body 유전자 중 **가장 강한 양의 상관 Top 5** 출력
3. 위치별 ρ 분포를 **KDE plot**으로 겹쳐 시각화

**힌트**: `cis_df[cis_df['location']=='Promoter'].nsmallest(5, 'rho')`, `sns.kdeplot()`

In [ ]:
# 미션 D2 코드를 여기에 작성하세요!

## 5. Top Cis-correlation 유전자

In [ ]:
# Top 양의/음의 cis-correlation
TOP_N = 10

top_neg = cis_df[cis_df['rho'] < 0].head(TOP_N)
top_pos = cis_df[cis_df['rho'] > 0].nlargest(TOP_N, 'rho')

print("=== Top 10 음의 cis-correlation (프로모터 침묵) ===")
for _, r in top_neg.iterrows():
    stars = sig_stars(r['fdr'])
    print(f"  {r['gene']:>12}  ρ={r['rho']:+.3f}  FDR={r['fdr']:.2e}  "
          f"{stars or 'ns'}  [{r['location']}]")

print("\n=== Top 10 양의 cis-correlation (유전자체 활성) ===")
for _, r in top_pos.iterrows():
    stars = sig_stars(r['fdr'])
    print(f"  {r['gene']:>12}  ρ={r['rho']:+.3f}  FDR={r['fdr']:.2e}  "
          f"{stars or 'ns'}  [{r['location']}]")

In [ ]:
# Top 유전자 산점도: 음의 상관 3개 + 양의 상관 3개
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle("Top Cis-correlation 유전자", fontsize=16, fontweight='bold')

def plot_cis_scatter(ax, gene, rho, loc, color):
    """유전자 하나의 발현 vs 메틸화 산점도 + 추세선"""
    e = mrna_cis.loc[gene].values.astype(float)
    m = meth_cis.loc[gene].values.astype(float)
    v = ~(np.isnan(e) | np.isnan(m))
    ax.scatter(e[v], m[v], alpha=0.6, s=40, color=color, edgecolors='white')
    # 추세선
    z = np.polyfit(e[v], m[v], 1)
    xl = np.linspace(e[v].min(), e[v].max(), 50)
    ax.plot(xl, np.poly1d(z)(xl), 'r--', alpha=0.7, lw=2)
    ax.set_xlabel(f"{gene} (mRNA)")
    ax.set_ylabel(f"{gene} (Methyl β)")
    ax.set_title(f"{gene} | ρ={rho:.3f} [{loc}]", fontsize=11)
    ax.grid(True, alpha=0.3)

# 상단: 음의 상관 Top 3
for i, (_, r) in enumerate(top_neg.head(3).iterrows()):
    plot_cis_scatter(axes[0, i], r['gene'], r['rho'], r['location'], 'blue')

# 하단: 양의 상관 Top 3
for i, (_, r) in enumerate(top_pos.head(3).iterrows()):
    plot_cis_scatter(axes[1, i], r['gene'], r['rho'], r['location'], 'red')

plt.tight_layout()
plt.show()

## 6. 아형별 Cis-correlation 비교

같은 유전자라도 암 아형에 따라 메틸화-발현 관계가 다를 수 있습니다.

In [ ]:
# 아형별 cis-correlation 비교 (Top 음의 상관 유전자 1개로 시연)
subtypes = clinical.set_index(SAMPLE_COL).loc[samples, 'Subtype']
main_subtypes = [s for s in subtypes.value_counts().index
                 if (subtypes == s).sum() >= 10][:4]

ex_gene = top_neg.iloc[0]['gene']  # Top 음의 상관 유전자
expr_all = mrna_cis.loc[ex_gene]
meth_all = meth_cis.loc[ex_gene]

# 전체 + 아형별 패널
n_panels = 1 + len(main_subtypes)
fig, axes = plt.subplots(1, n_panels, figsize=(5 * n_panels, 4.5))
fig.suptitle(f"{ex_gene}: 아형별 cis-correlation", fontsize=14, fontweight='bold')
panel_colors = ['steelblue', '#e74c3c', '#2ecc71', '#3498db', '#f39c12']

# 전체 산점도
v = ~(expr_all.isna() | meth_all.isna())
rho_all = spearmanr(expr_all[v], meth_all[v])[0]
axes[0].scatter(expr_all[v], meth_all[v], alpha=0.5, s=30, c=panel_colors[0])
axes[0].set_title(f"전체 (n={v.sum()})\nρ = {rho_all:.3f}")
axes[0].set_xlabel(f"{ex_gene} (mRNA)")
axes[0].set_ylabel(f"{ex_gene} (Methyl)")
axes[0].grid(True, alpha=0.3)

# 아형별 산점도
for i, st in enumerate(main_subtypes):
    mask = subtypes == st
    e_sub = expr_all[mask]
    m_sub = meth_all[mask]
    v_sub = ~(e_sub.isna() | m_sub.isna())
    rho = spearmanr(e_sub[v_sub], m_sub[v_sub])[0] if v_sub.sum() > 5 else np.nan
    
    axes[i+1].scatter(e_sub[v_sub], m_sub[v_sub], alpha=0.5, s=30, c=panel_colors[i+1])
    axes[i+1].set_title(f"{st} (n={v_sub.sum()})\nρ = {rho:.3f}")
    axes[i+1].set_xlabel(f"{ex_gene} (mRNA)")
    axes[i+1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 아형별 전체 cis-correlation 분포 비교
fig, ax = plt.subplots(figsize=(10, 5))

for st in main_subtypes:
    st_samples = subtypes[subtypes == st].index
    
    # 이 아형 내에서 모든 유전자의 ρ 계산
    rhos = []
    for gene in common_genes:
        e = mrna_cis.loc[gene, st_samples]
        m = meth_cis.loc[gene, st_samples]
        valid = ~(e.isna() | m.isna())
        if valid.sum() > 10:
            r, _ = spearmanr(e[valid], m[valid])
            rhos.append(r)
    
    sns.kdeplot(rhos, label=f'{st} (n={len(st_samples)})',
                ax=ax, fill=True, alpha=0.3)

ax.axvline(0, ls='--', c='gray', alpha=0.5)
ax.set_xlabel("Spearman ρ")
ax.set_ylabel("밀도")
ax.set_title("아형별 Cis-correlation 분포")
ax.legend()
plt.tight_layout()
plt.show()

### Vibe Coding 미션 D3: 아형별 유의 유전자 비교
**미션**: 암 아형에 따라 cis-correlation 패턴이 다른지 비교해보세요!

**결과물**: 아형별 유의 유전자 수 막대그래프

**도전 과제**:
1. 각 아형에서 **유의한 cis-correlation 유전자 수**를 세기
2. 아형별 유의 유전자 수를 **막대그래프**로 비교
3. (도전) 아형 간 ρ 차이가 가장 큰 유전자 1개의 산점도 그리기

**힌트**: 아형별로 `spearmanr()` 계산 → `p < 0.05`인 유전자 수 세기

In [ ]:
# 미션 D3 코드를 여기에 작성하세요!

### Vibe Coding 미션 D4: 종양억제유전자 후보 찾기
**미션**: 메틸화에 의해 침묵된 종양억제유전자 후보를 찾아보세요!

**결과물**: 후보 유전자 목록 표 (`pd.DataFrame` 출력)

**도전 과제**:
1. `cis_df`에서 **프로모터** + **ρ < -0.3** + **FDR < 0.05** 조건으로 필터링
2. 필터링 결과를 **DataFrame 표**로 깔끔하게 출력 (유전자명, ρ, FDR)
3. (도전) 후보 유전자 1개를 골라 **발현 vs 메틸화 산점도** 그리기

**힌트**: `cis_df[(cis_df['location']=='Promoter') & (cis_df['rho'] < -0.3) & (cis_df['fdr'] < 0.05)]`

In [ ]:
# 미션 D4 코드를 여기에 작성하세요!

### Vibe Coding 미션 D5: 아형별 메틸화 차이 시각화
**미션**: 암 아형 간 메틸화 수준이 다른 유전자를 찾아 시각화해보세요!

**결과물**: Top 유전자의 아형별 메틸화 박스플롯

**도전 과제**:
1. 아형별 메틸화 평균이 가장 다른 유전자 **Top 5** 찾기 (분산 기준 or Kruskal-Wallis)
2. Top 5 유전자의 **아형별 메틸화 박스플롯** 그리기
3. (도전) 같은 유전자의 아형별 **발현** 박스플롯과 나란히 비교

**힌트**: `meth_cis.loc['유전자명']` + `subtypes` 결합 → `sns.boxplot(x='Subtype', y='meth')`

In [ ]:
# 미션 D5 코드를 여기에 작성하세요!

## 7. Top 유전자 × 임상변수 연관

In [ ]:
# Top 음의 상관 유전자 × 임상변수 상관 히트맵
neg_genes = cis_df[cis_df['rho'] < 0].head(10)['gene'].values
clin_indexed = clinical.set_index(SAMPLE_COL).loc[samples]

# 주요 임상변수 선택
clinical_vars = ['Diagnosis Age', 'Fraction Genome Altered', 'Aneuploidy Score',
                 'TMB (nonsynonymous)', 'Mutation Count']
clinical_vars = [v for v in clinical_vars if v in clin_indexed.columns]

if clinical_vars and len(neg_genes) > 0:
    # 유전자 발현 × 임상변수 Spearman 상관 계산
    corr_mat = pd.DataFrame(index=neg_genes, columns=clinical_vars, dtype=float)
    pval_mat = pd.DataFrame(index=neg_genes, columns=clinical_vars, dtype=float)
    
    for gene in neg_genes:
        expr = mrna_cis.loc[gene]
        for var in clinical_vars:
            vals = pd.to_numeric(clin_indexed[var], errors='coerce')
            valid = expr.notna() & vals.notna()
            if valid.sum() > 10:
                r, p = spearmanr(expr[valid], vals[valid])
                corr_mat.loc[gene, var] = r
                pval_mat.loc[gene, var] = p

    # 히트맵 어노테이션 (상관계수 + 유의성 별표)
    annot = corr_mat.copy().astype(object)
    for r in corr_mat.index:
        for c in corr_mat.columns:
            v, p = corr_mat.loc[r, c], pval_mat.loc[r, c]
            annot.loc[r, c] = f'{v:.2f}{sig_stars(p)}' if pd.notna(v) else ''

    fig, ax = plt.subplots(figsize=(12, 6))
    sns.heatmap(corr_mat.astype(float), annot=annot.values, fmt='',
                cmap='RdBu_r', center=0, vmin=-0.5, vmax=0.5,
                linewidths=0.5, ax=ax)
    ax.set_title("Top 음의 cis-corr 유전자 × 임상변수\n(* p<0.05  ** p<0.01  *** p<0.001)")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha='right')
    plt.tight_layout()
    plt.show()

## 8. Cis-correlation 히트맵

유의한 유전자들의 발현 + 메틸화 패턴을 한 눈에 비교

In [ ]:
# 유의한 유전자 중 |ρ|가 큰 순서로 Top 20 선택
sig_genes = cis_df[cis_df['significant']].copy()
sig_genes['abs_rho'] = sig_genes['rho'].abs()
hm_genes = sig_genes.nlargest(20, 'abs_rho')['gene'].values

if len(hm_genes) < 10:
    hm_genes = cis_df.iloc[:20]['gene'].values

# 발현 + 메틸화 z-score
expr_hm = StandardScaler().fit_transform(mrna_cis.loc[hm_genes].T).T
meth_hm = StandardScaler().fit_transform(meth_cis.loc[hm_genes].T).T

# Cis-corr 사이드바
cis_vals = cis_df.set_index('gene').loc[hm_genes, 'rho'].values

fig, axes = plt.subplots(1, 2, figsize=(18, max(8, len(hm_genes)*0.3)),
                          gridspec_kw={'width_ratios': [1, 1]})

# 발현 히트맵
sns.heatmap(pd.DataFrame(expr_hm, index=hm_genes),
            cmap='RdBu_r', center=0, vmin=-3, vmax=3,
            xticklabels=False, yticklabels=True, ax=axes[0],
            cbar_kws={'label': 'z-score', 'shrink': 0.5})
axes[0].set_title('mRNA 발현 (z-score)', fontweight='bold')
axes[0].set_ylabel('')

# 메틸화 히트맵
sns.heatmap(pd.DataFrame(meth_hm, index=hm_genes),
            cmap='RdBu_r', center=0, vmin=-3, vmax=3,
            xticklabels=False, yticklabels=False, ax=axes[1],
            cbar_kws={'label': 'z-score', 'shrink': 0.5})
axes[1].set_title('Methylation (z-score)', fontweight='bold')

plt.suptitle(f'Top {len(hm_genes)} Cis-correlation 유전자: 발현 vs 메틸화',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 결과 저장

In [ ]:
import os
os.makedirs(RESULT_DIR, exist_ok=True)

# Cis-correlation 결과 저장
cis_df.to_csv(f"{RESULT_DIR}/nb03_cis_correlation_results.tsv", sep="\t", index=False)

# 유의한 유전자/프로브 목록 저장 (후속 분석용)
selected_genes = cis_df[cis_df['significant']]['gene'].tolist()
pd.Series(selected_genes).to_csv(f"{RESULT_DIR}/nb03_selected_genes.txt",
                                  index=False, header=False)

selected_probes = probe_info[probe_info['gene'].isin(selected_genes)]['probe'].unique()
pd.Series(selected_probes).to_csv(f"{RESULT_DIR}/nb03_selected_probes.txt",
                                   index=False, header=False)

n_sig = cis_df['significant'].sum()
print(f"저장 완료: {RESULT_DIR}/")
print(f"  nb03_cis_correlation_results.tsv ({len(cis_df)}개 유전자, 유의 {n_sig}개)")
print(f"  nb03_selected_genes.txt ({len(selected_genes)}개 유전자)")
print(f"  nb03_selected_probes.txt ({len(selected_probes)}개 프로브)")

---
## 분석 요약

| 단계 | 내용 |
|------|------|
| 데이터 준비 | Gene-level methylation (TSS 우선 집계) + mRNA 공통 유전자 매칭 |
| Cis-correlation | 공통 유전자별 mRNA 발현 ↔ 메틸화 Spearman 상관 |
| FDR 보정 | Benjamini-Hochberg (검정 수 = 공통 유전자 수) |
| 위치별 비교 | Promoter (음의 상관) vs Body (양의 상관) 패턴 확인 |
| 아형별 분석 | 아형에 따른 cis-correlation 차이 |
| 임상 연관 | Top 유전자 × 임상변수 Spearman 히트맵 |

### 핵심 발견
- **프로모터 메틸화 → 발현 억제** (음의 cis-correlation)
- **유전자체 메틸화 → 전사 활성** (양의 cis-correlation)
- Cis 방식으로 검정 수 대폭 감소 → 통계적 파워 향상